In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
def take_needed_cols(path, cols_from_modeller, cols2preparator, id2check=None):
    df = pd.read_csv(path, sep=';')
    
    assert len(cols_from_modeller) == len(cols2preparator)
    mod2prep = {key:val for key, val in zip(cols_from_modeller, cols2preparator)}
    df = df[cols_from_modeller]
    df = df.rename(columns=mod2prep)
    df = df.drop_duplicates()
    if id2check:
        assert df[id2check].duplicated().sum() == 0, 'ther are some duplicates in ids'
    return df

def clear_iri(text):
    match = re.findall('#.*', text)
    if match:
        res = match[0].replace('"', '').replace('#', '')
        return res

In [3]:
path = 'data_from_modeller/Arkadiy_Line_CN.2023-12-11.14_31_51.csv'
df_rs = pd.read_csv(path, sep=';')
cols_from_modeller = ['_id|R', 'IdentifiedObject.name|R', 
                      'class|R', 'ConnectivityNodeContainer.ConnectivityNodes|R', 
                      'EquipmentContainer.Equipments|R', 'Line.Region|R']
cols2preparator = ['IRI', 'name', 'Класс', 'ConnectivityNodeContainer','Equipments', 'Lines']

df_rs = take_needed_cols(path, cols_from_modeller, cols2preparator, id2check='IRI')
display(df_rs.head())
df_rs.info()
for col in df_rs.columns:
    print(col, df_rs[col].duplicated().sum())

df_rs.to_csv('output_data/Line/Line4preparator_full.csv', index=False)

,IRI,name,Класс,ConnectivityNodeContainer,Equipments,Lines
0,92979e0e-6929-47cc-a00a-b2ee6c294253,ВЛ 110 кВ Южная – Потеряевская с отпайками (ВЛ ЮГ-153),Line,"b9889929-1625-4e87-99aa-981c6243f46d, 55e85e6e-7c14-4057-9a3c-bd6b16abac10","f65210a5-22c6-478b-a980-0c9ddd2aa5e7, 6c2fc69f-a18f-4960-90fa-3f3a603eb05e, 381907f6-a3c9-4f11-a...",9a9a4278-260d-416d-a654-456ca29e9acd
10,7950f00d-abde-4e96-bd77-c7c82e99f845,ВЛ 110 кВ Панкрушихинская – Велижановская (ВЛ ПВ-4),Line,NaN,9fb70444-b533-441d-9c5b-b1beaa6690fa,9a9a4278-260d-416d-a654-456ca29e9acd
12,00332d8d-9a55-49fe-9d51-f0ded58a886f,ВЛ 110 кВ Бийская – Бийская ТЭЦ (ВЛ БТ-417),Line,NaN,5c319149-1e19-4429-b53f-c89f7cfe8ec4,9a9a4278-260d-416d-a654-456ca29e9acd
14,05e50330-f8d7-425b-967a-9ee1b67c56a0,КВЛ 110 кВ Барнаульская ТЭЦ-3 – Власиха II цепь (КВЛ ТВ-176),Line,NaN,1a177d91-fb14-4495-8301-82342f40d7f3,9a9a4278-260d-416d-a654-456ca29e9acd
16,d4fd9bc6-bc38-4b3c-b9a1-c484567099d2,ВЛ 110 кВ Солтонская – Ненинская (ВЛ СН-156),Line,NaN,4b7fdcc6-bb8b-46a1-b7e4-088ebec04197,9a9a4278-260d-416d-a654-456ca29e9acd


<class 'pandas.core.frame.DataFrame'>
Index: 946 entries, 0 to 5578
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   IRI                        946 non-null    object
 1   name                       946 non-null    object
 2   Класс                      946 non-null    object
 3   ConnectivityNodeContainer  493 non-null    object
 4   Equipments                 944 non-null    object
 5   Lines                      946 non-null    object
dtypes: object(6)
memory usage: 51.7+ KB
IRI 0
name 2
Класс 945
ConnectivityNodeContainer 452
Equipments 1
Lines 937


In [4]:
df_supa = pd.read_excel('data_supa/ТМ_ВЛ_КЛ110_20231221.xlsx')
df_supa = df_supa.dropna(axis=0, subset='Техместо')
df_supa['БЕ'] = df_supa['БЕ'].astype('int')
display(df_supa.head())
for col in df_supa.columns:
    print(col, df_supa[col].duplicated().sum())

,ГруппПолномочий,Техместо,Название,Дисп.наименование,Местоположение,НаименованиеМестоположения,ВерхнееТМ,Район,Класс,БЕ,ПланирующЗавод,Видобъекта,ОснСредство,ПользовСтатус,СистемнСтатус,Вэксплуат.с,Годвыпуска,Инвентарн№,НомерОСверх.ТМ
0,552A,KL110-000001,КЛ110кВ Фрунзенская-Прибрежная №1(С-105),КЛ 110 кВ Фрунзенская - Прибпержная (С-105),5.520000e+09,Западные Э/С,KL110-000001,NaN,LL,5500,5520.0,1033.0,T5500SRSK,ДСТВ,СОЗД,2010-12-31,NaN,NaN,T5500SRSK
1,2400,KL110-000002,Проект КЛ 110 кВ,NaN,2.400000e+09,Исполнительный аппарат ПАО Красэнерго,KL110-000002,NaN,LL,2400,2400.0,1033.0,T2400SRSK,СТРТ,СОЗД,NaT,NaN,NaN,T2400SRSK
2,552A,KL110-000003,КЛ110кВ Фрунзенская-Прибрежная №2(С-106),КЛ 110 кВ Фрунзенская - Прибрежная (С-106),5.520000e+09,Западные Э/С,KL110-000003,NaN,LL,5500,5520.0,1033.0,T5500SRSK,ДСТВ,СОЗД,2010-12-14,2010.0,NaN,T5500SRSK
3,244G,KL110-000004,МТКУ,NaN,2.440020e+09,Советский РЭС,KL110-000004,NaN,LL,2400,2440.0,1033.0,NaN,ДМНТ,СОЗД МТКУ,2007-12-29,NaN,Г000048383,NaN
4,4210,KL110-000005,КЛ-110 кВ,NaN,4.210000e+09,Северо-Восточные Э/С,KL110-000005,NaN,LL,4200,4210.0,1033.0,T4200FOBJ,ДМНТ,СОЗД МТКУ,2017-02-17,2017.0,NaN,T4200FOBJ


ГруппПолномочий 1294
Техместо 0
Название 15
Дисп.наименование 136
Местоположение 1298
НаименованиеМестоположения 1310
ВерхнееТМ 0
Район 1352
Класс 1352
БЕ 1345
ПланирующЗавод 1320
Видобъекта 1351
ОснСредство 825
ПользовСтатус 1346
СистемнСтатус 1350
Вэксплуат.с 1085
Годвыпуска 1283
Инвентарн№ 801
НомерОСверх.ТМ 825


In [5]:
be2region = {
    4200: 'Кемеровская область',
    2400: 'Красноярский край',
    2200: 'Алтайский край',
    300: 'Республика Бурятия',
    1900: 'Республика Хакасия',
    7500: 'Забайкальский край',
    400: 'Республика Алтай',
    5500: 'Омская область',
}

In [6]:
cnt = Counter(df_rs['name'].apply(lambda x: x.split(' ')[0]).tolist())
dict(cnt)

{'ВЛ': 894, 'КВЛ': 41, 'ПС': 5, 'КЛ': 6}

In [7]:
df_rs['first_sylls'] = df_rs['name'].apply(lambda x: x.split(' ')[0])
df_rs = df_rs[df_rs['first_sylls'].isin(['КВЛ', 'ВЛ'])].drop('first_sylls', axis=1)
df_rs = df_rs[~df_rs['name'].str.contains('35 кВ')]

In [8]:
filter = df_supa['Название'].str.contains('ВЛ-35кВ') | df_supa['Название'].str.contains('ВЛ-35 кВ') | df_supa['Дисп.наименование'].str.contains('ВЛ-35кВ') | df_supa['Дисп.наименование'].str.contains('ВЛ-35 кВ')
df_supa = df_supa[~filter]

In [9]:
df_supa['Название'] = df_supa['Название'].fillna('nan')
df_supa['Дисп.наименование'] = df_supa['Дисп.наименование'].fillna('nan')
df_rs['name'] = df_rs['name'].fillna('nan')

df_supa['Название'] = df_supa['Название'].apply(lambda x: x.replace('II', '2').replace('I', '1'))
df_supa['Дисп.наименование'] = df_supa['Дисп.наименование'].apply(lambda x: x.replace('II', '2').replace('I', '1'))
df_rs['name'] = df_rs['name'].apply(lambda x: x.replace('II', '2').replace('I', '1'))

In [10]:
# supa_duples_list = df_supa[df_supa['Дисп.наименование'].duplicated()]['Дисп.наименование'].unique().tolist()
# if 'nan' in supa_duples_list:
#     supa_duples_list.remove('nan')
# df_supa[df_supa['Дисп.наименование'].isin(supa_duples_list)].sort_values('Дисп.наименование')

In [11]:
# for dest in ['Камала', 'Дзержинская', 'Идринская', 'Тайшет',]:
#     print(dest)
#     display(df_supa[df_supa['Дисп.наименование'].str.contains(dest)])
#     print(df_supa[df_supa['Дисп.наименование'].str.contains(dest)]['Дисп.наименование'].unique())
#     display(df_rs[df_rs['name'].str.contains(dest)])
#     print(df_rs[df_rs['name'].str.contains(dest)]['name'].unique())

In [12]:
display(df_supa.info())
display(df_rs.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1337 entries, 0 to 1355
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ГруппПолномочий             1337 non-null   object        
 1   Техместо                    1337 non-null   object        
 2   Название                    1337 non-null   object        
 3   Дисп.наименование           1337 non-null   object        
 4   Местоположение              1337 non-null   float64       
 5   НаименованиеМестоположения  1337 non-null   object        
 6   ВерхнееТМ                   1337 non-null   object        
 7   Район                       0 non-null      float64       
 8   Класс                       1337 non-null   object        
 9   БЕ                          1337 non-null   int64         
 10  ПланирующЗавод              1337 non-null   float64       
 11  Видобъекта                  1337 non-null   float64       
 1

None

<class 'pandas.core.frame.DataFrame'>
Index: 931 entries, 0 to 5578
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   IRI                        931 non-null    object
 1   name                       931 non-null    object
 2   Класс                      931 non-null    object
 3   ConnectivityNodeContainer  490 non-null    object
 4   Equipments                 929 non-null    object
 5   Lines                      931 non-null    object
dtypes: object(6)
memory usage: 50.9+ KB


None

# Коды внутри названий

In [13]:
texts = [
    'ВЛ 110 кВ Бийская ТЭЦ – Сосна II цепь (ВЛ ТС-170)', 
    'ВЛ 110 кВ Бийская – Северо-Западная (ВЛ БС-59)',
    'ВЛ 110 кВ Бийская ТЭЦ – Сосна II цепь (ВЛ ТС-170)',
    'Бийская - Северо-Западная (  ВЛ БС-59  )',
    'ВЛ 110кВ "ПП Назаровский - Водозабор" (С-37)',
    np.nan
        ]

def find_code(text):
    if isinstance(text, str):
        match = re.findall('\(.*\)', text)
        if match:
            return match[0]\
                .replace('(', '')\
                .replace(')', '')\
                .replace(' ', '')\
                .replace('-', '')\
                .strip()
    return ''

for text in texts:
    print(text, '->', find_code(text))

ВЛ 110 кВ Бийская ТЭЦ – Сосна II цепь (ВЛ ТС-170) -> ВЛТС170
ВЛ 110 кВ Бийская – Северо-Западная (ВЛ БС-59) -> ВЛБС59
ВЛ 110 кВ Бийская ТЭЦ – Сосна II цепь (ВЛ ТС-170) -> ВЛТС170
Бийская - Северо-Западная (  ВЛ БС-59  ) -> ВЛБС59
ВЛ 110кВ "ПП Назаровский - Водозабор" (С-37) -> С37
nan -> 


In [14]:
df_supa['codes'] = df_supa['Дисп.наименование'].apply(find_code)
df_rs['codes'] = df_rs['name'].apply(find_code)

supa_codes_list = df_supa['codes'].tolist()
rs_codes_list = df_rs['codes'].tolist()

supa_codes_most_common50 = dict(Counter(supa_codes_list).most_common(50))
rs_codes_most_common50 = dict(Counter(rs_codes_list).most_common(50))

print('supa most_common codes', supa_codes_most_common50)
print('rs most_common codes', rs_codes_most_common50)

supa_codes = set(supa_codes_list)
rs_codes = set(rs_codes_list)

print('supa_codes & rs_codes =', len(supa_codes & rs_codes))

supa_codes_cnt = dict(Counter(supa_codes_list))
rs_codes_cnt = dict(Counter(rs_codes_list))

supa_codes_single = set([key for key in supa_codes_cnt.keys() if supa_codes_cnt[key] == 1])
rs_codes_single = set([key for key in rs_codes_cnt.keys() if rs_codes_cnt[key] == 1])
print('supa_codes_single & rs_codes_single', len(supa_codes_single & rs_codes_single))

supa most_common codes {'': 478, 'С72': 6, 'С73': 5, 'С90': 5, 'С38': 4, 'С36': 4, 'С5': 4, 'С75': 4, 'С96': 4, 'С68': 4, 'С67': 4, 'С24': 3, 'С98': 3, 'С85': 3, 'С81': 3, 'С46': 3, 'С25': 3, 'С37': 3, 'С79': 3, 'С71': 3, 'С43': 3, 'С23': 3, 'С26': 3, 'С80': 3, 'С70': 3, 'С8': 3, 'С6': 3, 'С123': 3, 'С128': 3, 'С124': 3, 'С65': 3, 'С167': 3, 'С120': 3, 'С135': 3, 'С82': 2, 'С87': 2, 'С88': 2, 'С89': 2, 'С83': 2, 'С55': 2, 'С58': 2, 'С62': 2, 'С32': 2, 'С34': 2, 'С66': 2, 'С811/Д35': 2, 'С814/С826': 2, 'С812/Д36': 2, 'С60': 2, 'С57': 2}
rs most_common codes {'': 295, 'С309': 2, 'С310': 2, 'эквивалент': 2, 'ВЛЮГ153': 1, 'ВЛПВ4': 1, 'ВЛБТ417': 1, 'КВЛТВ176': 1, 'ВЛСН156': 1, 'ВЛХЧ9': 1, 'ВЛСЛ191': 1, 'ВЛДК63': 1, 'ВЛСК72': 1, 'ВЛТС170': 1, 'КВЛТТ122': 1, 'ВЛГГ97': 1, 'КВЛВТ111': 1, 'ВЛЗО137': 1, 'ВЛБВ13': 1, 'ВЛЮВ151': 1, 'ВЛОЧ91': 1, 'ВЛКС116': 1, 'ВЛКТ186': 1, 'ВЛБЗ165': 1, 'ВЛБТ106': 1, 'ВЛАК19': 1, 'ВЛПК132': 1, 'ВЛЧЗ36': 1, 'ВЛКК114': 1, 'ВЛБС57': 1, 'ВЛЮГ148': 1, 'ВЛКК113': 1, 'ВЛТШ

# Названия откуда - куда

In [15]:
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
from io import BytesIO

g = tokenize(BytesIO('Бийская-Северо-Западная (  ВЛ БС-59  )'.encode('utf-8')).readline)
for toktype, tokval, _, _, _ in g:
    if toktype == NAME:
        print(tokval)

Бийская
Северо
Западная
ВЛ
БС


In [16]:
texts = [
    'ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170)', 
    'ВЛ 110 кВ Бийская – Северо-Западная (ВЛ БС-59)',
    'ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170)',
    'Бийская - Северо-Западная (  ВЛ БС-59  )',
    'Бийская-Северо-Западная (  ВЛ БС-59  )',
    'ВЛ 110 кВ Бийская–Северо-Западная 1 цепь (ВЛ БC-60 )',
    'ВЛ 110 кВ Бийская – Сосна 2 цепь с отпайками (ВЛ БС-58)',
    'КВЛ 110 кВ Бийская – Сосна 2 цепь с  отпайками на ПС (ВЛ БС-58)',
    'ВЛ 110кВ Дульдурга - Мордой 1 цепь с отпайкой на ПС Урейская (ВЛ-110-46)',
    'ВЛ 110 кВ Еланская - Хвостохранилище-2',
    'ВЛ 110 кВ Идринская - Н.Березовская с отпайкой на ПС Романовская (С-367/368)',
    'ВЛ 110кВ "ПП Назаровский - Водозабор" (С-37)',
    'ВЛ 110 кВ Ново-Кемеровская ТЭЦ-Азот-3',
    'ВЛ 110 кВ Азот-Азот -4 (потребительная)',
    'ВЛ 110 кВ Ново-Кемеровская ТЭЦ – Сотая-2',
    'ВЛ 110 кВ Еланская-Тальжино тяговая-1',
    'ВЛ 110 кВ КМК-1 – Опорная-6 1 цепь',
    'ВЛ 110 кВ КМК-1 – Опорная-6-2 с отпай',
    'ВЛ 110 кВ Идринская – Беллыкская с отпайкой на ПС Новая Сыда (С-96)',
    'ВЛ 110 кВ Беловская ГРЭС – Гурьевская ГРЭС 2 цепь с отпайкой на ПС Цинзаводская',
    np.nan
        ]

num2str = {
    '11': 'одиннадцать',
    '20': 'двадцать',
    '31': 'тридцать_один',
    '54': 'пятьдесят_четыре',
    '1': 'один',
    '2': 'два',
    '3': 'три',
    '4': 'четыре',
    '6': 'шесть',
}

def get_num(name, text):
    num = 0
    match = re.findall(f'{name} ?-? ?\d+', text)
    if match:
        num = re.findall('\d+', match[0])[0]
    text = re.sub(f'{name} ?№? ?-?\d', f'{name}-{num2str.get(num)}', text)
    return text

def find_destination(text):
    if isinstance(text, str):
        match = re.findall('\(.*\)', text)
        if match:
            text = text[:text.index(match[0])]
            
        text = get_num('Азот', text)
        text = get_num('Сотая', text)
        text = get_num('тяговая', text)
        text = get_num('Тяговая', text)
        text = get_num('Томусинская', text)
        text = get_num('Товарищ', text)
        text = get_num('Ширпотреб-Северная', text)
        text = get_num('Ширпотреб-Прокатная', text)
        text = get_num('Опорная', text)
        text = get_num('Тырганская', text)
        text = get_num('Абразивная', text)
        text = get_num('Распадская 5', text)
        text = get_num('Хвостохранилище', text)
        text = get_num('Карьерная', text)
        text = get_num('Разъезд', text)
        text = get_num('ЯЦЗ', text)
        text = get_num('Химпром', text)
        text = get_num('Проскоковская', text)
        text = get_num('Западно-Сибирская ТЭЦ', text)
        text = get_num('Западно-Сибирская ТЭЦ - Опорная', text)
        text = get_num('Ускатская-Луговая', text)
        text = get_num('Темирская', text)
        text = get_num('Шушталепская', text)
        text = get_num('Красноярская', text)
                       
        text = re.sub('ё', 'е', text)
        text = re.sub('с отп.на', '', text)
        text = re.sub('Цинзаводская', 'Цинкзаводская', text)
        text = re.sub('Заискиимская', 'Заискитимская', text)
        text = re.sub('Имена', 'Имени', text)
        text = re.sub('цепь', '', text)
        text = re.sub('Беловская ГРЭС – Гурьевская ГРЭС', 'Беловская ГРЭС – Гурьевская', text)
        text = re.sub('ЦентральнаяТЭЦ', 'Центральная ТЭЦ', text)
        text = re.sub('СибирскаяТЭЦ', 'Сибирская ТЭЦ', text)
        text = re.sub('[сc]{1}к\.', 'ский', text)
        text = re.sub('Беллык', 'Беллыкский', text)
        text = re.sub('[сc]{1}кая', 'ский', text)
        text = re.sub('Беллыкскийский', 'Беллыкский', text)
        text = re.sub('К?ВЛ[ -]110 ?кВ', '', text)
        text = re.sub('М.Имыш', 'Малый Имыш', text)
        text = re.sub('К.Шалтырь', 'Кия-Шалтырь', text)
        text = re.sub('Н.Огур', 'Новый Огур', text)
        text = re.sub('Н.Сыда', 'Новая Сыда', text)
        text = re.sub('Мусохраноская', 'Мусохрановская', text)
        text = re.sub('\d+ ?цепь', '', text)
        text = re.sub('[сc]? *отпай\w+ *(на)?', '', text)
        text = re.sub('[Н]?ПС', '', text)
        text = re.sub('ПП', '', text)
        text = text.lower()
        text = re.sub('обьект', 'объект', text)
        text = re.sub('"', '', text)
        tokens = []
        g = tokenize(BytesIO(text.encode('utf-8')).readline)
        for toktype, tokval, _, _, _ in g:
            if toktype == NAME:
                tokens.append(tokval.strip())
        return '_'.join(tokens).strip()
    return ''

for text in texts:
    print(text, '->', find_destination(text))

ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170) -> бийский_тэц_сосна
ВЛ 110 кВ Бийская – Северо-Западная (ВЛ БС-59) -> бийский_северо_западная
ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170) -> бийский_тэц_сосна
Бийская - Северо-Западная (  ВЛ БС-59  ) -> бийский_северо_западная
Бийская-Северо-Западная (  ВЛ БС-59  ) -> бийский_северо_западная
ВЛ 110 кВ Бийская–Северо-Западная 1 цепь (ВЛ БC-60 ) -> бийский_северо_западная
ВЛ 110 кВ Бийская – Сосна 2 цепь с отпайками (ВЛ БС-58) -> бийский_сосна
КВЛ 110 кВ Бийская – Сосна 2 цепь с  отпайками на ПС (ВЛ БС-58) -> бийский_сосна
ВЛ 110кВ Дульдурга - Мордой 1 цепь с отпайкой на ПС Урейская (ВЛ-110-46) -> дульдурга_мордой_урейский
ВЛ 110 кВ Еланская - Хвостохранилище-2 -> еланский_хвостохранилище_два
ВЛ 110 кВ Идринская - Н.Березовская с отпайкой на ПС Романовская (С-367/368) -> идринский_н_березовский_романовский
ВЛ 110кВ "ПП Назаровский - Водозабор" (С-37) -> назаровский_водозабор
ВЛ 110 кВ Ново-Кемеровская ТЭЦ-Азот-3 -> ново_кемеровский_т

In [17]:
df_supa['destinations'] = df_supa['Дисп.наименование'].apply(find_destination)
df_rs['destinations'] = df_rs['name'].apply(find_destination)

supa_dests_list = df_supa['destinations'].tolist()
rs_dests_list = df_rs['destinations'].tolist()

supa_dests_most_common50 = dict(Counter(supa_dests_list).most_common(50))
rs_dests_most_common50 = dict(Counter(rs_dests_list).most_common(50))

print('supa most_common dests', supa_dests_most_common50)
print('rs most_common dests', rs_dests_most_common50)

supa_dests = set(supa_dests_list)
rs_dests = set(rs_dests_list)

print('supa_dests & rs_dests =', len(supa_dests & rs_dests))

supa_dests_cnt = dict(Counter(supa_dests_list))
rs_dests_cnt = dict(Counter(rs_dests_list))

supa_dests_single = set([key for key in supa_dests_cnt.keys() if supa_dests_cnt[key] == 1])
rs_dests_single = set([key for key in rs_dests_cnt.keys() if rs_dests_cnt[key] == 1])
print('supa_dests_single & rs_dests_single', len(supa_dests_single & rs_dests_single))

supa most_common dests {'nan': 102, 'шп': 6, 'омв_кв': 5, 'сора_дзержинский': 4, 'барнаульский_тэц_власиха': 4, 'канский_опорная_камала_злмк': 4, 'омский_тэц_ск': 4, 'омский_тэц_густафьево': 4, 'дарасун_дульдурга': 4, 'абаканский_тэц_г_апов': 4, 'калининский_полярная': 4, 'кузнецкая_тэц_к': 4, 'темирский_казский': 4, 'центр_весна': 3, 'красноярский_тэц_заводский': 3, 'киск_миндерла': 3, 'омский_тэц_октябрьский': 3, 'резервное_техместо': 3, 'селенгинский_цкк_тимлюйский_кв_кабанский': 2, 'майминский_горно_алтайский': 2, 'канский_опорная_правобережная': 2, 'районная_машзавод': 2, 'районная_улан_удэнский_тэц': 2, 'районная_улан_удэнский_тэц_энергетик': 2, 'го_грэс_гусиноозерский': 2, 'акташский_кош_агачский': 2, 'акташский_улаганский': 2, 'ининский_акташский': 2, 'означенное_районная_г': 2, 'означенное_районная_стройбаза_г': 2, 'г_электрокотельная': 2, 'калининский_северная': 2, 'сора_боград': 2, 'медведчиково_северная': 2, 'гидролизная_насосная': 2, 'туим_шира': 2, 'смоленский_советский':

In [18]:
df_test_merge = df_rs[df_rs['codes'].isin(supa_codes_single & rs_codes_single)].merge(df_supa[df_supa['codes'].isin(supa_codes_single & rs_codes_single)], on='codes', suffixes=['_rs', '_supa'])[['IRI', 'name', 'Класс_rs',
       'Lines', 'codes', 'destinations_rs', 'Техместо', 'Название', 'Дисп.наименование',
       'НаименованиеМестоположения', 'ОснСредство', 'Класс_supa', 'destinations_supa']]
df_test_merge.shape[0]

524

In [19]:
df_test_merge['dests_equal'] = df_test_merge['destinations_rs'] == df_test_merge['destinations_supa']
df_test_merge[df_test_merge['dests_equal'] == False]

,IRI,name,Класс_rs,Lines,codes,destinations_rs,Техместо,Название,Дисп.наименование,НаименованиеМестоположения,ОснСредство,Класс_supa,destinations_supa,dests_equal
47,60309cdd-2aa0-4c0e-94f2-e01c8da47592,ВЛ 110 кВ Благовещенская - ТЭЦ КСК 1 цепь (ВЛ БК-157),Line,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛБК157,благовещенский_тэц_кск,VL110-001258,БК-157 Благовещенка-Кучукская,ВЛ 110 кВ Благовещенка-Кучукская (ВЛ БК-157),Родинский РЭС,T2200POTR,LL,благовещенка_кучукский,False
123,f7035e90-448f-41b3-bb69-60dece8365b8,ВЛ 110 кВ ТЭЦ АКХЗ - Камышенская с отпайкой на ПС Косихинская (ВЛ АК-78),Line,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛАК78,тэц_акхз_камышенский_косихинский,VL110-000219,АКХЗ-Косиха (ВЛ АК-78),ВЛ 110 кВ ТЭЦ АКХЗ-Косихинская с отпайкой на ПС Камышенская (ВЛ АК-78),Северо-Восточные Э/С,105010307,LL,тэц_акхз_косихинский_камышенский,False
125,a041b7ba-9e68-4750-84e0-270f051d31c9,ВЛ 110 кВ Тальменская – Тракторная (ВЛ ТТ-1411),Line,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛТТ1411,тальменский_тракторная,VL110-000313,Тальменская-Тракторная (ВЛ ТТ-1411),ВЛ 110 кВ Усть-Тальменская – Тракторная (ВЛ ТТ-1411),Северо-Восточные Э/С,T2200SRSK,LL,усть_тальменский_тракторная,False
126,94708710-60cf-426b-890c-293ae85edafb,ВЛ 110 кВ Тальменская - Новоеловская (ВЛ ТН-160),Line,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛТН160,тальменский_новоеловский,VL110-000308,Тальменская-Новоеловская (ВЛ ТН-160),ВЛ 110 кВ Усть-Тальменская – Новоеловская (ВЛ ТН-160),Северо-Восточные Э/С,105008537,LL,усть_тальменский_новоеловский,False
132,a38dd8a5-38a5-4ce4-b31e-71f1875db148,ВЛ 110 кВ Славгородская – Бурлинская с отпайкой на ПС Бурсоль (ВЛ СБ-128),Line,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛСБ128,славгородский_бурлинский_бурсоль,VL110-000248,Славгородская-Бурлинская (ВЛ СБ-128),ВЛ 110 кВ Славгородская-Бурлинская (ВЛ СБ-128),Кулундинские Э/С,105003669,LL,славгородский_бурлинский,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,b1a7d0c7-f180-49d4-9ccf-47cd9775b376,ВЛ 110 кВ БУР-1 - Жилпоселок с отпайкой на ПС Конвейерного транспорта (С-766),Line,1909ccf4-ef38-42ae-9d8b-dcc141888710,С766,бур_жилпоселок_конвейерного_транспорта,VL110-000442,ВЛ 110кВ БУР1-Жилпоселок с отп. (С-766),ВЛ 110кВ БУР1-Жилпоселок 2цепь с отпайкой на ПС Конвейерный транспорт (С-766),ПО Западные электрические сети,T2400SRSK,LL,бур1_жилпоселок_конвейерный_транспорт,False
460,f5773433-f8bb-4973-a7e9-a4a5f92355f6,ВЛ 110 кВ Шарыповская - Западная с отпайкой на ПС Береш (С-757),Line,1909ccf4-ef38-42ae-9d8b-dcc141888710,С757,шарыповский_западная_береш,VL110-000354,ВЛ 110кВ Шарыповская-Береш с отп.(С-757),ВЛ 110кВ Шарыповская-Береш 1 цепь с отпайкой на ПС Западная (С-757),ПО Западные электрические сети,T2400SRSK,LL,шарыповский_береш_западная,False
474,85509a7b-557f-4b18-8751-6f2510802c83,ВЛ 110 кВ Вершина Шахтамы – Ново-Широкая с отпайкой на ПС Быстринская (ВЛ-110-28),Line,eda583fc-3b4f-42fb-a871-9b09664a129b,ВЛ11028,вершина_шахтамы_ново_широкая_быстринский,VL110-001102,ВЛ-110-28 В.Шахтамы-Н.Широкинск,ВЛ 110 кВ Вершина Шахтамы - Ново-Широкая с отпайкой на ПС Бугдаинская (ВЛ-110-28),Юго-Восточные Э/С,105001642,LL,вершина_шахтамы_ново_широкая_бугдаинский,False
480,f6ddbe0c-311d-4c40-ab17-d3e85bfb5978,ВЛ 110 кВ Дульдурга – Мордой с отпайками (ВЛ-110-46),Line,eda583fc-3b4f-42fb-a871-9b09664a129b,ВЛ11046,дульдурга_мордой,VL110-001103,ВЛ 110 кВ Дульдурга-Мордой 1 цепь с отпа,ВЛ 110 кВ Дульдурга - Мордой 1 цепь с отпайкой на ПС Урейск (ВЛ-110-46),Юго-Западные Э/С,105002263,LL,дульдурга_мордой_урейск,False


# codes + destinations

In [20]:
df_supa['dests_plus_codes'] = df_supa['destinations']+'_'+df_supa['codes']
df_rs['dests_plus_codes'] = df_rs['destinations']+'_'+df_rs['codes']

# ручное
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ ТЭЦ АКХЗ-Городская (ВЛ АГ-88) 2 цепь с отпайкой на ПС Кокс (АК-1436)'), 'dests_plus_codes'] = 'тэц_акхз_городский_кокс_ВЛАГ88,ВЛАК1436'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ ТЭЦ АКХЗ – Городская (ВЛ АГ-87) 1 цепь с отпайкой на ПС Кокс (ВЛ АК-1435)'), 'dests_plus_codes'] = 'тэц_акхз_городский_кокс_ВЛАГ87,ВЛАК1435'
# ---

supa_dests_plus_codes_list = df_supa['dests_plus_codes'].tolist()
rs_dests_plus_codes_list = df_rs['dests_plus_codes'].tolist()

supa_dests_plus_codes_most_common50 = dict(Counter(supa_dests_plus_codes_list).most_common(50))
rs_dests_plus_codes_most_common50 = dict(Counter(rs_dests_plus_codes_list).most_common(50))

print('supa most_common dests_plus_codes', supa_dests_plus_codes_most_common50)
print('rs most_common dests_plus_codes', rs_dests_plus_codes_most_common50)

supa_dests_plus_codes = set(supa_dests_plus_codes_list)
rs_dests_plus_codes = set(rs_dests_plus_codes_list)

print('supa_dests_plus_codes & rs_dests_plus_codes =', len(supa_dests_plus_codes & rs_dests_plus_codes))

supa_dests_plus_codes_cnt = dict(Counter(supa_dests_plus_codes_list))
rs_dests_plus_codes_cnt = dict(Counter(rs_dests_plus_codes_list))

supa_dests_plus_codes_single = set([key for key in supa_dests_plus_codes_cnt.keys() if supa_dests_plus_codes_cnt[key] == 1])
rs_dests_plus_codes_single = set([key for key in rs_dests_plus_codes_cnt.keys() if rs_dests_plus_codes_cnt[key] == 1])
common_dests_plus_codes_single = supa_dests_plus_codes_single & rs_dests_plus_codes_single
print('supa_dests_plus_codes_single & rs_dests_plus_codes_single', len(common_dests_plus_codes_single))

supa most_common dests_plus_codes {'nan_': 102, 'шп_': 6, 'омв_кв_': 4, 'темирский_казский_': 4, 'резервное_техместо_': 3, 'южная_приозерная_': 2, 'барнаульский_тэц_власиха_': 2, 'барнаульский_тэц_подгорная_кристалл_': 2, 'кварцит_тяговая_тайшет_запань_тяговая_С46': 2, 'канский_опорная_камала_злмк_С811/Д35': 2, 'канский_опорная_дзержинский_С814/С826': 2, 'канский_опорная_камала_злмк_С812/Д36': 2, 'абакумовка_тяговая_тайшет_запань_тяговая_С43': 2, 'киск_миндерла_С226/С236': 2, 'анжерский_ново_анжерский_': 2, 'ново_анжерский_анжерский_': 2, 'юргинский_яшкинский_': 2, 'идринский_романовский_н_березовский_С367/368': 2, 'заискитимский_мирная_заводский_': 2, 'заискитимский_оросительная_водозабор_': 2, 'крохалевский_кедровский_': 2, 'кемеровский_ново_кемеровский_тэц_': 2, 'кемеровский_топкинский_': 2, 'крохалевский_черниговский_': 2, 'кемеровский_южная_': 2, 'тяжинский_каштан_тяговая_': 2, 'кемеровский_грэс_рудничная_': 2, 'заискитимский_восточная_имени_лапина_в_и_': 2, 'беловский_беловский_г

In [21]:
df_merge1 = df_rs[df_rs['dests_plus_codes'].isin(rs_dests_plus_codes_single)].merge(df_supa[df_supa['dests_plus_codes'].isin(supa_dests_plus_codes_single)], on='dests_plus_codes', suffixes=['_rs', '_supa'])
df_merge1.shape[0]

685

In [22]:
for col in df_merge1.columns:
    print(col, df_merge1[col].duplicated().sum())
    assert df_merge1[col].duplicated().sum() == 0 if col in ['IRI', 'Техместо'] else True

IRI 0
name 0
Класс_rs 684
ConnectivityNodeContainer 335
Equipments 1
Lines 676
codes_rs 151
destinations_rs 118
dests_plus_codes 0
ГруппПолномочий 654
Техместо 0
Название 0
Дисп.наименование 0
Местоположение 654
НаименованиеМестоположения 662
ВерхнееТМ 0
Район 684
Класс_supa 684
БЕ 678
ПланирующЗавод 658
Видобъекта 684
ОснСредство 308
ПользовСтатус 682
СистемнСтатус 684
Вэксплуат.с 467
Годвыпуска 625
Инвентарн№ 295
НомерОСверх.ТМ 308
codes_supa 151
destinations_supa 118


In [23]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_merge1['Техместо'].unique())]
df_rs = df_rs[~df_rs['IRI'].isin(df_merge1['IRI'].unique())]
df_supa.shape[0], df_rs.shape[0]

(652, 246)

In [24]:
df_merge2 = df_rs[df_rs['codes'].isin(supa_codes_single & rs_codes_single)].merge(df_supa[df_supa['codes'].isin(supa_codes_single & rs_codes_single)], on='codes', suffixes=['_rs', '_supa'])
df_merge2.shape[0]

63

In [25]:
for col in df_merge2.columns:
    print(col, df_merge2[col].duplicated().sum())
    assert df_merge2[col].duplicated().sum() == 0 if col in ['IRI', 'Техместо'] else True

IRI 0
name 0
Класс_rs 62
ConnectivityNodeContainer 22
Equipments 0
Lines 57
codes 0
destinations_rs 10
dests_plus_codes_rs 0
ГруппПолномочий 45
Техместо 0
Название 0
Дисп.наименование 0
Местоположение 45
НаименованиеМестоположения 46
ВерхнееТМ 0
Район 62
Класс_supa 62
БЕ 57
ПланирующЗавод 46
Видобъекта 62
ОснСредство 20
ПользовСтатус 62
СистемнСтатус 62
Вэксплуат.с 20
Годвыпуска 40
Инвентарн№ 23
НомерОСверх.ТМ 20
destinations_supa 11
dests_plus_codes_supa 0


In [26]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_merge2['Техместо'].unique())]
df_rs = df_rs[~df_rs['IRI'].isin(df_merge2['IRI'].unique())]
df_supa.shape[0], df_rs.shape[0]

(589, 183)

In [27]:
# from pprint import pprint
# pprint(list(zip(df_merge2['name'].tolist(), df_merge2['Дисп.наименование'].tolist())))

In [28]:
print((df_merge1.shape[0] + df_merge2.shape[0])/(df_supa.shape[0] + df_merge1.shape[0] + df_merge2.shape[0]))
(df_merge1.shape[0] + df_merge2.shape[0])/(df_rs.shape[0] + df_merge1.shape[0] + df_merge2.shape[0])

0.5594614809274495


0.80343716433942

# цепь

In [29]:
texts = [
    'ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170)', 
    'ВЛ 110 кВ Бийская – Северо-Западная (ВЛ БС-59)',
    'ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170)',
    'Бийская - Северо-Западная (  ВЛ БС-59  )',
    'Бийская-Северо-Западная (  ВЛ БС-59  )',
    'ВЛ 110 кВ Бийская–Северо-Западная 1 цепь (ВЛ БC-60 )',
    'ВЛ 110 кВ Бийская – Сосна 2 цепь с отпайками (ВЛ БС-58)',
    'КВЛ 110 кВ Бийская – Сосна 2 цепь с  отпайками на ПС (ВЛ БС-58)',
    'ВЛ 110кВ Дульдурга - Мордой 1 цепь с отпайкой на ПС Урейская (ВЛ-110-46)',
    'ВЛ 110 кВ Ново-Анжерская-Анжерская НПС-1',
    'ВЛ 110 кВ Ново-Анжерская-Анжерская НПС-2',
    'ВЛ 110 кВ Ново-Анжерская-Спутник-1',
    'ВЛ 110 кВ Ново-Анжерская-Спутник-2',
    'ВЛ 110 кВ КМК-1 – Опорная-6 1 цепь',
    np.nan
        ]

def find_chain(text):
    if isinstance(text, str):
        text = text.replace('Опорная 11-1', 'Опорная-11 1 цепь')
        text = text.replace('Опорная 11-2', 'Опорная-11 2 цепь')
        text = text.replace('Опорная 20-1', 'Опорная 20 1 цепь')
        text = text.replace('Опорная 20-2', 'Опорная 20 2 цепь')
        text = re.sub('НПС-1', '1 цепь', text)
        text = re.sub('НПС-2', '2 цепь', text)
        text = re.sub('Спутник-1', 'Спутник 1 цепь', text)
        text = re.sub('Спутник-2', 'Спутник 2 цепь', text)
        match = re.findall('\d цепь', text)
        if match:
            res = match[0].split(' ')
            return res[0]
    return '0'

for text in texts:
    print(text, '->', find_chain(text))

ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170) -> 2
ВЛ 110 кВ Бийская – Северо-Западная (ВЛ БС-59) -> 0
ВЛ 110 кВ Бийская ТЭЦ – Сосна 2 цепь (ВЛ ТС-170) -> 2
Бийская - Северо-Западная (  ВЛ БС-59  ) -> 0
Бийская-Северо-Западная (  ВЛ БС-59  ) -> 0
ВЛ 110 кВ Бийская–Северо-Западная 1 цепь (ВЛ БC-60 ) -> 1
ВЛ 110 кВ Бийская – Сосна 2 цепь с отпайками (ВЛ БС-58) -> 2
КВЛ 110 кВ Бийская – Сосна 2 цепь с  отпайками на ПС (ВЛ БС-58) -> 2
ВЛ 110кВ Дульдурга - Мордой 1 цепь с отпайкой на ПС Урейская (ВЛ-110-46) -> 1
ВЛ 110 кВ Ново-Анжерская-Анжерская НПС-1 -> 1
ВЛ 110 кВ Ново-Анжерская-Анжерская НПС-2 -> 2
ВЛ 110 кВ Ново-Анжерская-Спутник-1 -> 1
ВЛ 110 кВ Ново-Анжерская-Спутник-2 -> 2
ВЛ 110 кВ КМК-1 – Опорная-6 1 цепь -> 1
nan -> 0


In [30]:
df_supa['chain'] = df_supa['Дисп.наименование'].apply(find_chain)
df_rs['chain'] = df_rs['name'].apply(find_chain)

df_supa['dests_plus_chains'] = df_supa['destinations']+'_'+df_supa['chain']
df_rs['dests_plus_chains'] = df_rs['destinations']+'_'+df_rs['chain']

# ручное
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Краснополянская – Красноярская № 2 с отпайками'), 'dests_plus_chains'] = 'краснополянский_красноярский_2'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Краснополянская – Красноярская № 1 с отпайками'), 'dests_plus_chains'] = 'краснополянский_красноярский_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Краснополянская – Красноярская 1 цепь сапйками'), 'dests_plus_chains'] = 'краснополянский_красноярский_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Краснополянская – Красноярская 2 цепь с отпайками'), 'dests_plus_chains'] = 'краснополянский_красноярский_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Прокопьевская-Кис-Заводская-1'), 'dests_plus_chains'] = 'прокопьевский_киселевский_заводский_вахрушевский_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Прокопьевская-Кис-Заводская-2'), 'dests_plus_chains'] = 'прокопьевский_киселевский_заводский_вахрушевский_2'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Томь-Усинская ГРЭС-Распадская 5-1'), 'dests_plus_chains'] = 'томь_усинский_грэс_распадский_пять_1'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Томь-Усинская ГРЭС-Распадская 5-2'), 'dests_plus_chains'] = 'томь_усинский_грэс_распадский_пять_2'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Томь-Усинская ГРЭС – ЦОФ-1'), 'dests_plus_chains'] = 'томь_усинский_грэс_цоф_1'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Томь-Усинская ГРЭС – ЦОФ-2'), 'dests_plus_chains'] = 'томь_усинский_грэс_цоф_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Томь-Усинская ГРЭС-Распадская 5-1'), 'dests_plus_chains'] = 'томь_усинский_грэс_распадский_пять_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Томь-Усинская ГРЭС-Распадская 5-2'), 'dests_plus_chains'] = 'томь_усинский_грэс_распадский_пять_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Томьусинская ГРЭС-ЦОФ Сибирь-1'), 'dests_plus_chains'] = 'томь_усинский_грэс_цоф_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Томьусинская ГРЭС-ЦОФ Сибирь-2'), 'dests_plus_chains'] = 'томь_усинский_грэс_цоф_2'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Ускатская - Луговая-1'), 'dests_plus_chains'] = 'ускатский_луговая_1'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Ускатская - Луговая-2'), 'dests_plus_chains'] = 'ускатский_луговая_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Ускатская-Луговая-1'), 'dests_plus_chains'] = 'ускатский_луговая_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Ускатская-Луговая-2'), 'dests_plus_chains'] = 'ускатский_луговая_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Ускатская-Ильинская-1 2 цепь'), 'dests_plus_chains'] = 'ускатский_ильинский_степная_2'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Западно-Сибирская ТЭЦ - Опорная - 20 1 цепь'), 'dests_plus_chains'] = 'западно_сибирский_тэц_опорная_двадцать0_1'
df_rs.loc[(df_rs['name'] == 'ВЛ 110 кВ Западно-Сибирская ТЭЦ - Опорная - 20 2 цепь'), 'dests_plus_chains'] = 'западно_сибирский_тэц_опорная_двадцать0_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Ширпотреб-Прокатная-1'), 'dests_plus_chains'] = 'ширпотреб_прокатная_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Ширпотреб-Прокатная-2'), 'dests_plus_chains'] = 'ширпотреб_прокатная_2'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Южно-Кузбаская ГРЭС-Кедровая-1'), 'dests_plus_chains'] = 'южно_кузбасский_грэс_кедровая_малиновский_1'
df_supa.loc[(df_supa['Дисп.наименование'] == 'ВЛ 110 кВ Южно-Кузбасская ГРЭС-Кедровая-2'), 'dests_plus_chains'] = 'южно_кузбасский_грэс_кедровая_малиновский_2'
# ---

supa_dests_plus_chains_list = df_supa['dests_plus_chains'].tolist()
rs_dests_plus_chains_list = df_rs['dests_plus_chains'].tolist()

supa_dests_plus_chains_most_common50 = dict(Counter(supa_dests_plus_chains_list).most_common(50))
rs_dests_plus_chains_most_common50 = dict(Counter(rs_dests_plus_chains_list).most_common(50))

print('supa most_common dests_plus_chains', supa_dests_plus_chains_most_common50)
print('rs most_common dests_plus_chains', rs_dests_plus_chains_most_common50)

supa_dests_plus_chains = set(supa_dests_plus_chains_list)
rs_dests_plus_chains = set(rs_dests_plus_chains_list)

print('supa_dests_plus_chains & rs_dests_plus_chains =', len(supa_dests_plus_chains & rs_dests_plus_chains))

supa_dests_plus_chains_cnt = dict(Counter(supa_dests_plus_chains_list))
rs_dests_plus_chains_cnt = dict(Counter(rs_dests_plus_chains_list))

supa_dests_plus_chains_single = set([key for key in supa_dests_plus_chains_cnt.keys() if supa_dests_plus_chains_cnt[key] == 1])
rs_dests_plus_chains_single = set([key for key in rs_dests_plus_chains_cnt.keys() if rs_dests_plus_chains_cnt[key] == 1])
print('supa_dests_plus_chains_single & rs_dests_plus_chains_single', len(supa_dests_plus_chains_single & rs_dests_plus_chains_single))

supa most_common dests_plus_chains {'nan_0': 102, 'шп_0': 6, 'омв_кв_0': 5, 'омский_тэц_ск_0': 4, 'калининский_полярная_0': 4, 'кузнецкая_тэц_к_0': 4, 'темирский_казский_0': 4, 'резервное_техместо_0': 3, 'кварцит_тяговая_тайшет_запань_тяговая_0': 2, 'бур_0': 2, 'канский_опорная_камала_злмк_1': 2, 'канский_опорная_дзержинский_0': 2, 'канский_опорная_камала_злмк_2': 2, 'абакумовка_тяговая_тайшет_запань_тяговая_0': 2, 'киск_миндерла_2': 2, 'идринский_романовский_н_березовский_0': 2, 'кемеровский_топкинский_0': 2, 'тяжинский_каштан_тяговая_0': 2, 'зсмк_опорная_три_0': 2, 'зсмк_опорная_none_0': 2, 'зсмк_северобайдаевский_0': 2, 'мариинск_каштан_тяговая_0': 2, 'кмк_опорная_три_0': 2, 'еланский_хвостохранилище_один_0': 2, 'еланский_хвостохранилище_два_0': 2, 'зсмк_опорная_четыре_0': 2, 'иртышский_иртышский_т_0': 2, 'кмк_опорная_шесть_0': 2, 'омский_тэц_икар_0': 2, 'омский_тэц_ульяновский_0': 2, 'омский_тэц_густафьево_1': 2, 'московка_обв_0': 2, 'петрушенко_новомарьяновский_0': 2, 'называевски

In [31]:
df_merge3 = df_rs[df_rs['dests_plus_chains'].isin(supa_dests_plus_chains_single & rs_dests_plus_chains_single)].merge(df_supa[df_supa['dests_plus_chains'].isin(supa_dests_plus_chains_single & rs_dests_plus_chains_single)], on='dests_plus_chains', suffixes=['_rs', '_supa'])
df_merge3.shape[0]

117

In [32]:
for col in df_merge3.columns:
    print(col, df_merge3[col].duplicated().sum())
    assert df_merge3[col].duplicated().sum() == 0 if col in ['IRI', 'Техместо'] else True

IRI 0
name 0
Класс_rs 116
ConnectivityNodeContainer 48
Equipments 0
Lines 110
codes_rs 109
destinations_rs 50
dests_plus_codes_rs 49
chain_rs 114
dests_plus_chains 0
ГруппПолномочий 101
Техместо 0
Название 0
Дисп.наименование 0
Местоположение 102
НаименованиеМестоположения 106
ВерхнееТМ 0
Район 116
Класс_supa 116
БЕ 110
ПланирующЗавод 104
Видобъекта 116
ОснСредство 43
ПользовСтатус 115
СистемнСтатус 116
Вэксплуат.с 62
Годвыпуска 78
Инвентарн№ 47
НомерОСверх.ТМ 43
codes_supa 103
destinations_supa 47
dests_plus_codes_supa 46
chain_supa 114


In [33]:
df_supa = df_supa[~df_supa['Техместо'].isin(df_merge3['Техместо'].unique())]
df_rs = df_rs[~df_rs['IRI'].isin(df_merge3['IRI'].unique())]
df_supa.shape[0], df_rs.shape[0]

(472, 66)

In [34]:
print((df_merge1.shape[0] + df_merge2.shape[0] + df_merge3.shape[0])/(df_supa.shape[0] + df_merge1.shape[0] + df_merge2.shape[0] + df_merge3.shape[0]))
print((df_merge1.shape[0] + df_merge2.shape[0] + df_merge3.shape[0])/(df_rs.shape[0] + df_merge1.shape[0] + df_merge2.shape[0] + df_merge3.shape[0]))

0.6469708302169035
0.9291084854994629


In [35]:
needed_cols = ['IRI', 'Техместо', 'name', 'Дисп.наименование', 'Класс_rs']
columns2rename = {'Класс_rs': 'Класс', 
                 'name': 'Наименование',
                 'Дисп.наименование': 'Диспетчерское наименование'}
df_union = pd.concat([df_merge1[needed_cols], df_merge2[needed_cols], df_merge3[needed_cols]]).rename(columns=columns2rename)
for col in df_union.columns:
    print(col, df_union[col].duplicated().sum())
print(df_union.shape)
display(df_union.info())
display(df_union.head())

filename = 'output_data/Line/Line_result.csv'
print(f'test saved {filename}')
df_union.to_csv(filename, index=False)
display(pd.read_csv(filename).head())
pd.read_csv(filename).info()

IRI 0
Техместо 0
Наименование 0
Диспетчерское наименование 0
Класс 864
(865, 5)
<class 'pandas.core.frame.DataFrame'>
Index: 865 entries, 0 to 116
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   IRI                         865 non-null    object
 1   Техместо                    865 non-null    object
 2   Наименование                865 non-null    object
 3   Диспетчерское наименование  865 non-null    object
 4   Класс                       865 non-null    object
dtypes: object(5)
memory usage: 40.5+ KB


None

,IRI,Техместо,Наименование,Диспетчерское наименование,Класс
0,92979e0e-6929-47cc-a00a-b2ee6c294253,VL110-000005,ВЛ 110 кВ Южная – Потеряевская с отпайками (ВЛ ЮГ-153),ВЛ 110 кВ Южная – Потеряевская с отпайками (ВЛ ЮГ-153),Line
1,7950f00d-abde-4e96-bd77-c7c82e99f845,VL110-000210,ВЛ 110 кВ Панкрушихинская – Велижановская (ВЛ ПВ-4),ВЛ 110 кВ Панкрушихинская – Велижановская (ВЛ ПВ-4),Line
2,00332d8d-9a55-49fe-9d51-f0ded58a886f,VL110-000129,ВЛ 110 кВ Бийская – Бийская ТЭЦ (ВЛ БТ-417),ВЛ 110 кВ Бийская – Бийская ТЭЦ (ВЛ БТ-417),Line
3,05e50330-f8d7-425b-967a-9ee1b67c56a0,VL110-000326,КВЛ 110 кВ Барнаульская ТЭЦ-3 – Власиха 2 цепь (КВЛ ТВ-176),КВЛ 110 кВ Барнаульская ТЭЦ-3 – Власиха 2 цепь (КВЛ ТВ-176),Line
4,d4fd9bc6-bc38-4b3c-b9a1-c484567099d2,VL110-000138,ВЛ 110 кВ Солтонская – Ненинская (ВЛ СН-156),ВЛ 110 кВ Солтонская – Ненинская (ВЛ СН-156),Line


test saved output_data/Line/Line_result.csv


,IRI,Техместо,Наименование,Диспетчерское наименование,Класс
0,92979e0e-6929-47cc-a00a-b2ee6c294253,VL110-000005,ВЛ 110 кВ Южная – Потеряевская с отпайками (ВЛ ЮГ-153),ВЛ 110 кВ Южная – Потеряевская с отпайками (ВЛ ЮГ-153),Line
1,7950f00d-abde-4e96-bd77-c7c82e99f845,VL110-000210,ВЛ 110 кВ Панкрушихинская – Велижановская (ВЛ ПВ-4),ВЛ 110 кВ Панкрушихинская – Велижановская (ВЛ ПВ-4),Line
2,00332d8d-9a55-49fe-9d51-f0ded58a886f,VL110-000129,ВЛ 110 кВ Бийская – Бийская ТЭЦ (ВЛ БТ-417),ВЛ 110 кВ Бийская – Бийская ТЭЦ (ВЛ БТ-417),Line
3,05e50330-f8d7-425b-967a-9ee1b67c56a0,VL110-000326,КВЛ 110 кВ Барнаульская ТЭЦ-3 – Власиха 2 цепь (КВЛ ТВ-176),КВЛ 110 кВ Барнаульская ТЭЦ-3 – Власиха 2 цепь (КВЛ ТВ-176),Line
4,d4fd9bc6-bc38-4b3c-b9a1-c484567099d2,VL110-000138,ВЛ 110 кВ Солтонская – Ненинская (ВЛ СН-156),ВЛ 110 кВ Солтонская – Ненинская (ВЛ СН-156),Line


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   IRI                         865 non-null    object
 1   Техместо                    865 non-null    object
 2   Наименование                865 non-null    object
 3   Диспетчерское наименование  865 non-null    object
 4   Класс                       865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


In [36]:
filename = 'output_data/Line/Line4preparator_unmatched.csv'
print(f'test saved {filename}')
df_rs.to_csv(filename, index=False)
display(pd.read_csv(filename).head())
pd.read_csv(filename).info()

test saved output_data/Line/Line4preparator_unmatched.csv


,IRI,name,Класс,ConnectivityNodeContainer,Equipments,Lines,codes,destinations,dests_plus_codes,chain,dests_plus_chains
0,25b7ae42-016d-4cd7-ac2c-28d2ea7b570a,ВЛ 110 кВ Благовещенская - ТЭЦ КСК 2 цепь (ВЛ БК-158),Line,NaN,22e0dea3-d12e-4b80-9f0a-e3ee57d674b1,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛБК158,благовещенский_тэц_кск,благовещенский_тэц_кск_ВЛБК158,2,благовещенский_тэц_кск_2
1,5e07f09f-6f18-4da6-bc81-1b400c7cc808,ВЛ 110 кВ Щербакты – Кулунда (Л-126/1),Line,"3c9d6778-53ac-4380-b693-4b60e4d9af62, 36ca8c53-c323-45d5-802c-fdcc6d1e8c87","f9e4b60a-9df6-41ab-a309-41f1f626f397, a9ebc8fe-688c-4719-961a-51eeaac072a7, 47f0c020-b464-4578-9...",9a9a4278-260d-416d-a654-456ca29e9acd,Л126/1,щербакты_кулунда,щербакты_кулунда_Л126/1,0,щербакты_кулунда_0
2,2542e868-2127-42b5-bbd8-e6da28936491,ВЛ 110 кВ ОПП – Майминская (ВЛ ОМ-139),Line,NaN,cecae105-8c03-4dc5-ac97-52ed7f3a5b94,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛОМ139,о_майминский,о_майминский_ВЛОМ139,0,о_майминский_0
3,dfbe7017-794b-4885-a698-b8b2069ccbae,ВЛ 110 кВ Дмитриевская – Ненинская (ВЛ ДН-86),Line,NaN,a931cf6b-e780-4b78-a684-7baf5bf98365,9a9a4278-260d-416d-a654-456ca29e9acd,ВЛДН86,дмитриевский_ненинский,дмитриевский_ненинский_ВЛДН86,0,дмитриевский_ненинский_0
4,1a06b56a-721e-4557-b38f-d5a6f411320d,ВЛ 110 кВ Мариинск – Каштан тяговая с отпайками,Line,"b3e40122-059e-4f93-9459-15cb0e9cd1ae, 08529e24-1926-418b-93c2-e8b3ed716860","a3108fed-5ddd-4d97-a8fe-b981b7af596c, 8d458e6f-7f06-446f-9344-de35079019b2, 1e9090ee-df2a-4d91-b...",d4d8ad78-244f-4408-8882-ceaf42460d27,NaN,мариинск_каштан_тяговая,мариинск_каштан_тяговая_,0,мариинск_каштан_тяговая_0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   IRI                        66 non-null     object
 1   name                       66 non-null     object
 2   Класс                      66 non-null     object
 3   ConnectivityNodeContainer  33 non-null     object
 4   Equipments                 66 non-null     object
 5   Lines                      66 non-null     object
 6   codes                      33 non-null     object
 7   destinations               66 non-null     object
 8   dests_plus_codes           66 non-null     object
 9   chain                      66 non-null     int64 
 10  dests_plus_chains          66 non-null     object
dtypes: int64(1), object(10)
memory usage: 5.8+ KB


In [37]:
# df_supa['chain'] = df_supa['Дисп.наименование'].apply(find_chain)
# df_rs['chain'] = df_rs['name'].apply(find_chain)

# df_supa['dests_plus_codes_and_chains'] = df_supa['destinations']+'_'+df_supa['codes']+'_'+df_supa['chain']
# df_rs['dests_plus_codes_and_chains'] = df_rs['destinations']+'_'+df_rs['codes']+'_'+df_rs['chain']


# supa_dests_plus_codes_and_chains_list = df_supa['dests_plus_codes_and_chains'].tolist()
# rs_dests_plus_codes_and_chains_list = df_rs['dests_plus_codes_and_chains'].tolist()

# supa_dests_plus_codes_and_chains_most_common50 = dict(Counter(supa_dests_plus_codes_and_chains_list).most_common(50))
# rs_dests_plus_codes_and_chains_most_common50 = dict(Counter(rs_dests_plus_codes_and_chains_list).most_common(50))

# print('supa most_common dests_plus_codes_and_chains', supa_dests_plus_codes_and_chains_most_common50)
# print('rs most_common dests_plus_codes_and_chains', rs_dests_plus_codes_and_chains_most_common50)

# supa_dests_plus_codes_and_chains = set(supa_dests_plus_codes_and_chains_list)
# rs_dests_plus_codes_and_chains = set(rs_dests_plus_codes_and_chains_list)

# print('supa_dests_plus_codes_and_chains & rs_dests_plus_codes_and_chains =', len(supa_dests_plus_codes_and_chains & rs_dests_plus_codes_and_chains))

# supa_dests_plus_codes_and_chains_cnt = dict(Counter(supa_dests_plus_codes_and_chains_list))
# rs_dests_plus_codes_and_chains_cnt = dict(Counter(rs_dests_plus_codes_and_chains_list))

# supa_dests_plus_codes_and_chains_single = set([key for key in supa_dests_plus_codes_and_chains_cnt.keys() if supa_dests_plus_codes_and_chains_cnt[key] == 1])
# rs_dests_plus_codes_and_chains_single = set([key for key in rs_dests_plus_codes_and_chains_cnt.keys() if rs_dests_plus_codes_and_chains_cnt[key] == 1])
# print('supa_dests_plus_codes_and_chains_single & rs_dests_plus_codes_and_chains_single', len(supa_dests_plus_codes_and_chains_single & rs_dests_plus_codes_and_chains_single))

In [38]:
# df_supa['chain'] = df_supa['Дисп.наименование'].apply(find_chain)
# df_rs['chain'] = df_rs['name'].apply(find_chain)

# df_supa['codes_plus_chains'] = df_supa['codes']+'_'+df_supa['chain']
# df_rs['codes_plus_chains'] = df_rs['codes']+'_'+df_rs['chain']


# supa_codes_plus_chains_list = df_supa['codes_plus_chains'].tolist()
# rs_codes_plus_chains_list = df_rs['codes_plus_chains'].tolist()

# supa_codes_plus_chains_most_common50 = dict(Counter(supa_codes_plus_chains_list).most_common(50))
# rs_codes_plus_chains_most_common50 = dict(Counter(rs_codes_plus_chains_list).most_common(50))

# print('supa most_common codes_plus_chains', supa_codes_plus_chains_most_common50)
# print('rs most_common codes_plus_chains', rs_codes_plus_chains_most_common50)

# supa_codes_plus_chains = set(supa_codes_plus_chains_list)
# rs_codes_plus_chains = set(rs_codes_plus_chains_list)

# print('supa_codes_plus_chains & rs_codes_plus_chains =', len(supa_codes_plus_chains & rs_codes_plus_chains))

# supa_codes_plus_chains_cnt = dict(Counter(supa_codes_plus_chains_list))
# rs_codes_plus_chains_cnt = dict(Counter(rs_codes_plus_chains_list))

# supa_codes_plus_chains_single = set([key for key in supa_codes_plus_chains_cnt.keys() if supa_codes_plus_chains_cnt[key] == 1])
# rs_codes_plus_chains_single = set([key for key in rs_codes_plus_chains_cnt.keys() if rs_codes_plus_chains_cnt[key] == 1])
# print('supa_codes_plus_chains_single & rs_codes_plus_chains_single', len(supa_codes_plus_chains_single & rs_codes_plus_chains_single))

In [39]:
exclude_destinations = ['благовещенский', 'тэц', 'кск', 'щербакты', 'кулунда', 'о',
                       'майминский', 'дмитриевский', 'ненинский', 'мариинск', 'каштан',
                       'тяговая', 'северный', 'маганак', 'базовая', 'еланский', 'хвостохранилище',
                       'один', 'два', 'заискиимский', 'в', 'и', 'тяжинский', 'степная']

for destination in df_rs['destinations']:
    tokens = destination.split('_')
    for token in tokens:
        if token not in exclude_destinations:
            print(token)
            print('rs')
            display(df_rs[df_rs['destinations'].str.contains(token)])
            print('supa')
            display(df_supa[df_supa['destinations'].str.contains(token)])

зсмк
rs


,IRI,name,Класс,ConnectivityNodeContainer,Equipments,Lines,codes,destinations,dests_plus_codes,chain,dests_plus_chains
2063,cabdebbc-8073-42e3-8b16-474abe42cd76,ВЛ 110 кВ ЗСМК-Северо-Байдаевская-1,Line,NaN,0217cb8e-e62c-4bb9-9ce6-6376f5d53a44,d4d8ad78-244f-4408-8882-ceaf42460d27,,зсмк_северо_байдаевский,зсмк_северо_байдаевский_,0,зсмк_северо_байдаевский_0
2075,a1afd97b-edd1-4b5c-8422-48d7ccd12018,ВЛ 110 кВ ЗСМК-Северо-Байдаевская-2,Line,NaN,48970eb9-84c6-4860-8a12-26a315d86290,d4d8ad78-244f-4408-8882-ceaf42460d27,,зсмк_северо_байдаевский,зсмк_северо_байдаевский_,0,зсмк_северо_байдаевский_0


supa


,ГруппПолномочий,Техместо,Название,Дисп.наименование,Местоположение,НаименованиеМестоположения,ВерхнееТМ,Район,Класс,БЕ,ПланирующЗавод,Видобъекта,ОснСредство,ПользовСтатус,СистемнСтатус,Вэксплуат.с,Годвыпуска,Инвентарн№,НомерОСверх.ТМ,codes,destinations,dests_plus_codes,chain,dests_plus_chains
678,4230,VL110-000672,ВЛ 110 кВ ЗСМК-Карлык тяговая-1,ВЛ 110 кВ ЗСМК – Карлык тяговая – 1,4.230000e+09,Южные Э/С,VL110-000672,NaN,LL,4200,4230.0,1031.0,105003114,ДСТВ,СОЗД,1968-01-01,1968.0,Д000015134,105003114,,зсмк_карлык_тяговая,зсмк_карлык_тяговая_,0,зсмк_карлык_тяговая_0
681,4230,VL110-000675,ВЛ 110 кВ ЗСМК-Опорная 3-2,ВЛ 110 кВ ЗСМК-Опорная 3-2,4.230000e+09,Южные Э/С,VL110-000675,NaN,LL,4200,4230.0,1031.0,105003113,ДСТВ,СОЗД,1968-01-01,1968.0,Д000006741,105003113,,зсмк_опорная_три,зсмк_опорная_три_,0,зсмк_опорная_три_0
684,4230,VL110-000678,ВЛ 110 кВ ЗСМК-Опорная 5-1,ВЛ 110 кВ ЗСМК-Опорная 5-1,4.230000e+09,Южные Э/С,VL110-000678,NaN,LL,4200,4230.0,1031.0,105003125,ДСТВ,СОЗД,1964-01-01,1964.0,Д000006771,105003125,,зсмк_опорная_none,зсмк_опорная_none_,0,зсмк_опорная_none_0
687,4230,VL110-000681,ВЛ 110 кВ ЗСМК-Северо-Байдаевская-2,ВЛ 110 кВ ЗСМК-Северобайдаевская-2,4.230000e+09,Южные Э/С,VL110-000681,NaN,LL,4200,4230.0,1031.0,105003117,ДСТВ,СОЗД,1960-01-01,1960.0,Д000006745,105003117,,зсмк_северобайдаевский,зсмк_северобайдаевский_,0,зсмк_северобайдаевский_0
755,4230,VL110-000749,ВЛ 110 кВ ЗСМК-Карлык тяговая-2,ВЛ 110 кВ ЗСМК-Карлык тяговая-2,4.230000e+09,Южные Э/С,VL110-000749,NaN,LL,4200,4230.0,1031.0,T4200SRSK,ДСТВ,СОЗД,1968-01-01,1968.0,NaN,T4200SRSK,,зсмк_карлык_тяговая_два,зсмк_карлык_тяговая_два_,0,зсмк_карлык_тяговая_два_0
758,4230,VL110-000752,ВЛ 110 кВ ЗСМК-Опорная 4-1,ВЛ 110 кВ ЗСМК-Опорная 4-1,4.230000e+09,Южные Э/С,VL110-000752,NaN,LL,4200,4230.0,1031.0,105003116,ДСТВ,СОЗД,1970-01-01,1970.0,Д000006744,105003116,,зсмк_опорная_четыре,зсмк_опорная_четыре_,0,зсмк_опорная_четыре_0
760,4230,VL110-000754,ВЛ 110 кВ ЗСМК-Опорная 3-1,ВЛ 110 кВ ЗСМК-Опорная 3-1,4.230000e+09,Южные Э/С,VL110-000754,NaN,LL,4200,4230.0,1031.0,T4200SRSK,ДСТВ,СОЗД,1968-01-01,1968.0,NaN,T4200SRSK,,зсмк_опорная_три,зсмк_опорная_три_,0,зсмк_опорная_три_0
761,4230,VL110-000755,ВЛ 110 кВ ЗСМК-Опорная 5-2,ВЛ 110 кВ ЗСМК-Опорная 5-2,4.230000e+09,Южные Э/С,VL110-000755,NaN,LL,4200,4230.0,1031.0,T4200SRSK,ДСТВ,СОЗД,1964-01-01,1964.0,NaN,T4200SRSK,,зсмк_опорная_none,зсмк_опорная_none_,0,зсмк_опорная_none_0
763,4230,VL110-000757,ВЛ 110 кВ ЗСМК-Опорная 4-2,ВЛ 110 кВ ЗСМК-Опорная 4-2,4.230000e+09,Южные Э/С,VL110-000757,NaN,LL,4200,4230.0,1031.0,T4200SRSK,ДСТВ,СОЗД,1970-01-01,1970.0,NaN,T4200SRSK,,зсмк_опорная_четыре,зсмк_опорная_четыре_,0,зсмк_опорная_четыре_0
766,4230,VL110-000760,ВЛ 110 кВ ЗСМК-Северо-Байдаевская-1,ВЛ 110 кВ ЗСМК-Северобайдаевская-1,4.230000e+09,Южные Э/С,VL110-000760,NaN,LL,4200,4230.0,1031.0,T4200SRSK,ДСТВ,СОЗД,1960-01-01,1960.0,NaN,T4200SRSK,,зсмк_северобайдаевский,зсмк_северобайдаевский_,0,зсмк_северобайдаевский_0


северо
rs


...

,ГруппПолномочий,Техместо,Название,Дисп.наименование,Местоположение,НаименованиеМестоположения,ВерхнееТМ,Район,Класс,БЕ,ПланирующЗавод,Видобъекта,ОснСредство,ПользовСтатус,СистемнСтатус,Вэксплуат.с,Годвыпуска,Инвентарн№,НомерОСверх.ТМ,codes,destinations,dests_plus_codes,chain,dests_plus_chains
953,7540,VL110-001017,ВЛ 110кВ Дарасун – Дульдурга (ВЛ-110-40),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 1 цепь (ВЛ-110-40;44),7.540000e+09,Центральные Э/С,VL110-001017,NaN,LL,7500,7540.0,1031.0,105000929,ДСТВ,СОЗД,1972-01-01,NaN,И000007176,105000929,ВЛ11040;44,дарасун_дульдурга,дарасун_дульдурга_ВЛ11040;44,1,дарасун_дульдурга_1
954,7540,VL110-001020,ВЛ 110кВ Дарасун – Дульдурга (ВЛ-110-43),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 2 цепь (ВЛ-110-43;45),7.540000e+09,Центральные Э/С,VL110-001020,NaN,LL,7500,7540.0,1031.0,105000923,ДСТВ,СОЗД,1972-01-01,NaN,И000007179,105000923,ВЛ11043;45,дарасун_дульдурга,дарасун_дульдурга_ВЛ11043;45,2,дарасун_дульдурга_2
989,7540,VL110-001087,Курорт Дарасун-Дульдурга (ВЛ-110-44),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 1 цепь (ВЛ-110-40;44),7.540000e+09,Центральные Э/С,VL110-001087,NaN,LL,7500,7540.0,1031.0,105000926,ДСТВ,СОЗД,1969-01-01,NaN,И000007181,105000926,ВЛ11040;44,дарасун_дульдурга,дарасун_дульдурга_ВЛ11040;44,1,дарасун_дульдурга_1
992,7540,VL110-001090,Курорт Дарасун-Дульдурга (ВЛ-110-45),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 2 цепь (ВЛ-110-43;45),7.540000e+09,Центральные Э/С,VL110-001090,NaN,LL,7500,7540.0,1031.0,T7500SRSK,ДСТВ,СОЗД,1969-01-01,NaN,И000007181/003,T7500SRSK,ВЛ11043;45,дарасун_дульдурга,дарасун_дульдурга_ВЛ11043;45,2,дарасун_дульдурга_2


дульдурга
rs


,IRI,name,Класс,ConnectivityNodeContainer,Equipments,Lines,codes,destinations,dests_plus_codes,chain,dests_plus_chains
5091,5b489076-ca95-4d2d-8591-f8b51aa0d829,"ВЛ 110 кВ Дарасун – Дульдурга 1 цепь с отпайками (ВЛ-110-40, 44)",Line,"bbb19840-1b53-4fb9-a149-3482b14428ab, 479beeab-907d-42cd-ad7f-118a8bc79430","fbb94ddc-00f1-4982-b5e4-3158bc360c3e, f784f617-c559-4b5a-8128-a664ac6385eb, e36b74f6-fb2d-4714-a...",eda583fc-3b4f-42fb-a871-9b09664a129b,"ВЛ11040,44",дарасун_дульдурга,"дарасун_дульдурга_ВЛ11040,44",1,дарасун_дульдурга_1
5106,a0fbf48e-b3eb-4ac8-858a-071d10d00079,"ВЛ 110 кВ Дарасун – Дульдурга 2 цепь с отпайками (ВЛ-110-43, 45)",Line,"d36fb541-32c6-49f1-b2dd-6faa354e60f1, 7ff73a0b-8670-4766-9e60-48b8dd45b7c3","f4d3634c-255c-40a8-a4ec-cf62cfa3cb40, da5baf2f-e6f8-4ca1-9364-9df323878422, c627c6b9-f608-4f9b-a...",eda583fc-3b4f-42fb-a871-9b09664a129b,"ВЛ11043,45",дарасун_дульдурга,"дарасун_дульдурга_ВЛ11043,45",2,дарасун_дульдурга_2


supa


,ГруппПолномочий,Техместо,Название,Дисп.наименование,Местоположение,НаименованиеМестоположения,ВерхнееТМ,Район,Класс,БЕ,ПланирующЗавод,Видобъекта,ОснСредство,ПользовСтатус,СистемнСтатус,Вэксплуат.с,Годвыпуска,Инвентарн№,НомерОСверх.ТМ,codes,destinations,dests_plus_codes,chain,dests_plus_chains
953,7540,VL110-001017,ВЛ 110кВ Дарасун – Дульдурга (ВЛ-110-40),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 1 цепь (ВЛ-110-40;44),7.540000e+09,Центральные Э/С,VL110-001017,NaN,LL,7500,7540.0,1031.0,105000929,ДСТВ,СОЗД,1972-01-01,NaN,И000007176,105000929,ВЛ11040;44,дарасун_дульдурга,дарасун_дульдурга_ВЛ11040;44,1,дарасун_дульдурга_1
954,7540,VL110-001020,ВЛ 110кВ Дарасун – Дульдурга (ВЛ-110-43),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 2 цепь (ВЛ-110-43;45),7.540000e+09,Центральные Э/С,VL110-001020,NaN,LL,7500,7540.0,1031.0,105000923,ДСТВ,СОЗД,1972-01-01,NaN,И000007179,105000923,ВЛ11043;45,дарасун_дульдурга,дарасун_дульдурга_ВЛ11043;45,2,дарасун_дульдурга_2
989,7540,VL110-001087,Курорт Дарасун-Дульдурга (ВЛ-110-44),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 1 цепь (ВЛ-110-40;44),7.540000e+09,Центральные Э/С,VL110-001087,NaN,LL,7500,7540.0,1031.0,105000926,ДСТВ,СОЗД,1969-01-01,NaN,И000007181,105000926,ВЛ11040;44,дарасун_дульдурга,дарасун_дульдурга_ВЛ11040;44,1,дарасун_дульдурга_1
992,7540,VL110-001090,Курорт Дарасун-Дульдурга (ВЛ-110-45),ВЛ 110 кВ Дарасун-Дульдурга с отпайками 2 цепь (ВЛ-110-43;45),7.540000e+09,Центральные Э/С,VL110-001090,NaN,LL,7500,7540.0,1031.0,T7500SRSK,ДСТВ,СОЗД,1969-01-01,NaN,И000007181/003,T7500SRSK,ВЛ11043;45,дарасун_дульдурга,дарасун_дульдурга_ВЛ11043;45,2,дарасун_дульдурга_2
